In [3]:
# !pip install -U -q "langchain" "transformers==4.31.0" "datasets==2.13.0" "peft==0.4.0" "accelerate==0.21.0" "bitsandbytes==0.40.2" "trl==0.4.7" "safetensors>=0.3.1"


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Data Prep

In [8]:
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/oppenheimer.csv

--2023-12-12 10:40:32--  https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/oppenheimer.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80957 (79K) [text/plain]
Saving to: 'oppenheimer.csv.2'

     0K .......... .......... .......... .......... .......... 63%  256K 0s
    50K .......... .......... .........                       100% 99.6M=0.2s

2023-12-12 10:40:33 (405 KB/s) - 'oppenheimer.csv.2' saved [80957/80957]



In [5]:
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/barbie.csv

--2023-12-12 10:39:43--  https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/barbie.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 72289 (71K) [text/plain]
Saving to: 'barbie.csv.1'

     0K .......... .......... .......... .......... .......... 70%  311K 0s
    50K .......... ..........                                 100%  108M=0.2s

2023-12-12 10:39:44 (438 KB/s) - 'barbie.csv.1' saved [72289/72289]



## Parse Data

In [9]:
from langchain.document_loaders.csv_loader import CSVLoader

ImportError: cannot import name 'stringify_dict' from 'langchain_core.utils' (c:\Users\TurnerZ\Documents\GitHub\DBSage_Navigator\.venv\Lib\site-packages\langchain_core\utils\__init__.py)

In [ ]:
barbie_loader = CSVLoader(file_path="barbie.csv", source_column="Review_Url", encoding="utf-8")
oppenheimer_loader = CSVLoader(file_path="oppenheimer.csv", source_column="Review_Url", encoding="utf-8")

In [ ]:
barbie_data = barbie_loader.load()
oppenheimer_data = oppenheimer_loader.load()

## Chunking Data

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000, # the character length of the chunk
    chunk_overlap = 100, # the character length of the overlap between chunks
    length_function = len, # the length function - in this case, character length (aka the python len() fn.)
)

In [ ]:
barbie_documents = text_splitter.transform_documents(barbie_data)
len(barbie_documents)

181

In [ ]:
oppenheimer_documents = text_splitter.transform_documents(oppenheimer_data)
len(oppenheimer_documents)

186

In [ ]:
combined_documents = barbie_documents + oppenheimer_documents

## Creating Index

In [1]:
!pip install -q -U faiss-cpu tiktoken sentence-transformers

  DEPRECATION: sentence-transformers is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559

[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from langchain.embeddings import CacheBackedEmbeddings, HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore

store = LocalFileStore("./cache/")

embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

core_embeddings_model = HuggingFaceEmbeddings(
    model_name=embed_model_id
)

embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings_model, store, namespace=embed_model_id
)

vector_store = FAISS.from_documents(combined_documents, embedder)

c:\Users\TurnerZ\Documents\GitHub\DBSage_Navigator\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\TurnerZ\Documents\GitHub\DBSage_Navigator\.venv\Lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


NameError: name 'combined_documents' is not defined